<a href="https://colab.research.google.com/github/przemek-c/robotyka/blob/test/kinematyka_odwrotna_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.colab
%pip install roboticstoolbox-python>=1.0.2
%pip install colored==1.4.4

In [ ]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%matplotlib notebook

Rozwiązanie zagadnienia kinematyki odwrotnej, podobnie jak zagadnienia kinematyki prostej, trzeba rozpocząć od stworzenia modelu manipulatora.

In [ ]:
L1 = rtb.DHLink(d=0.645, alpha=pi/2, theta=0.0, a=0.330,)
L2 = rtb.DHLink(d=0.0, alpha=0.0, theta=0.0, a=1.35)
L3 = rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.115)
L4= rtb.DHLink(d=1.420, alpha=-pi/2, theta=0.0, a=0.0)
L5= rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.0)
L6= rtb.DHLink(d=0.215, alpha=0.0, theta=0.0, a=0.0)
robot = rtb.DHRobot([L1, L2, L3, L4, L5, L6])

W następnym kroku konieczne jest określenie położenia i orientacji końcówki manipulatora, dla której zagadnienie  ma zostać rozwiązane. Pozycję tą należny przedstawić w postaci macierzy homogenicznej. Jednym z najprostszych sposobów jest określenie macierzy translacji i rotacji. <br>
Macierz translacji stworzymy obliczając ją dla przykładu z kinematyki prostej. W ten sposób manipulator ustawi się ostatecznie w tym samym miejscu.

Rozwiązanie rozwiązania zagadnienia kinematyki odwrotnej można otrzymać wywołując na obiekcie robota metodę **ikine_LM** i podając jako argument macierz jednorodną.
```py
sol = robot.ikine_LM(T)
```
W zwracanym obiekcie *sol* zawarte są informacje o rozwiązaniu problemu, w tym współrzędne złączowe i informacja o tym czy udało się osiągnąć zadaną pozycję.

In [ ]:
i = 19
new_q = [pi/125*i, pi/2+(-pi/225*i), -pi/100*i, -pi/50*i, -pi/100*i, -pi/100*i]
T = robot.fkine(new_q)
print(T)
sol = robot.ikine_LM(T)
sol

  -0.3013    0.7579    0.5787    1.63      
   0.7932    0.536    -0.289     0.717     
  -0.5292    0.3719   -0.7626    0.7805    
   0         0         0         1         



IKSolution(q=array([  -2.664,    2.229,    2.972,  -0.8133,   0.8028,    2.054]), success=True, iterations=15, searches=1, residual=1.2619959472657988e-11, reason='Success')

In [ ]:
trans = SE3(-1, 1, 2)
y = [0,0,1]
z = [1,0,0]
rot = SE3.OA(y, z)
T = trans * rot
T
sol = robot.ikine_LM(T)
print("Without mask: ")
print(sol)
mask = np.array([1, 1, 1, 0, 0, 0])
sol = robot.ikine_LM(T, mask=mask)
print("With mask: ")
print(sol)

Without mask: 
IKSolution: q=[-0.6886, 1.936, 2.63, -1.747, 2.44, 2.913], success=True, iterations=11, searches=1, residual=2.4e-09
With mask: 
IKSolution: q=[2.263, 1.959, -0.3839, 1.45, -0.667, 1.624], success=True, iterations=8, searches=1, residual=4.38e-09


iterujac po czasie <br>
obliczam nowe sol dla kolejnego T<br>
łączę całość

In [ ]:
# punkty T w postaci listy
# interpolate in python:
# https://docs.scipy.org/doc/scipy/tutorial/interpolate/1D.html
# Cubic splines, 1st derivative

my_trans_co = [
               (0, 0, 0),
               (-1, 1, 2),
               (-1, -1, 2),
               (1, -1, 2),
               (1, 1, 2)]


In [ ]:
def calculate_sol(x):
  trans = SE3(x)
  y = [0,0,1]
  z = [1,0,0]
  rot = SE3.OA(y, z)
  T = trans * rot
  mask = np.array([1, 1, 1, 0, 0, 0])
  sol = robot.ikine_LM(T, mask=mask)
  if sol.success == True:
    return sol.q


In [ ]:
# od zera do len(T)
qr = []
# my_time = 0.0
for i in range(len(my_trans_co)):
  # my_time = my_time + 0.1
  my_sol = calculate_sol(my_trans_co[i])
  # qr.append(my_time)
  qr.append(my_sol)
q_array = np.array(qr)
print(q_array)

[[-0.09681  -0.2987   -1.191   -1.757        0    2.648]
 [ -0.8011    2.981     0.43    2.333    2.998  -0.6276]
 [   -2.37  0.09509   -3.012       -3    2.454   0.6755]
 [ -0.8416  0.08545    3.093  -0.6592    2.494   -1.415]
 [  0.6622    1.757  -0.3014   -1.212      2.1    0.293]]


In [ ]:
robot.plot(q_array, movie="robot_ikine.gif")

PyPlot3D backend, t = 0.25, scene:
  robot: Text(0.0, 0.0, '')

In [ ]:
print(sol.success)
print(sol.q)

True
[  0.4775    1.306  -0.5969   -1.194  -0.5969  -0.5969]


In [ ]:
with open("wynik.txt", "w") as plik:
  for q in q_array:
    print(q)
    plik.write(','.join(map(str,q)))
    plik.write('\n')

[-0.09681  -0.2987   -1.191   -1.757        0    2.648]
[ -0.8011    2.981     0.43    2.333    2.998  -0.6276]
[   -2.37  0.09509   -3.012       -3    2.454   0.6755]
[ -0.8416  0.08545    3.093  -0.6592    2.494   -1.415]
[  0.6622    1.757  -0.3014   -1.212      2.1    0.293]


In [ ]:
import pandas as pd
read_file = pd.read_csv ('wynik.txt')
read_file.to_csv ('wynik_w_csv.csv', index=None)
df = pd.read_csv('wynik_w_csv.csv')
# dodajemy naglowki
df.columns =['q1','q2','q3','q4','q5','q6']
# jaka dlugosc csv
rows_in_csv = df.shape[0]
# do takiej wartosci for tworzy liste ze skokiem 0.1
tt = []
for x in range(0, rows_in_csv):
  t = x*0.1
  tt.append(t)
# dodajemy to jako pierwsza kolumna
df['time'] = tt
df = df[['time', 'q1','q2','q3','q4','q5','q6']]
df.to_csv('updated_wynik_w_csv.csv', index=False)

First need path planning:
found nothing in google <br>
Need to figure out on my own:
muszę założyć że już znam punkty,
jak auto wjeżdża na podnośnik to zatrzymuje się w konkretnym miejscu.
Wtedy wiem że spaw idzie na wysokości np 1.2 m i idze pod kątem 30
So generate the workspace first
czyli znam te punkty i one będą w postaci macierzy T
to teraz od jednego do drugiego
Joint vs Task się pojawia (oba dochodzą do punktu)
zróbmy Task czyli jak mam dwa punkty to jak wygenerować te pośrednie?
czyli Interpolacja?
spline itp
na wykresie wartosc kazdej zmiennej w T i do tego czas
ale czy ja je moge zmieniac osobno?
https://www.youtube.com/watch?v=Fd7wjZDoh7g
trapezoidcal, polynomial trajectories like cubic sounds good
